In [ ]:
The dataset utilized here is available on Kaggle. 
https://www.kaggle.com/datasets/gvclsu/water-segmentation-dataset

Resize the Data

In [ ]:
#resize image
import os, sys
from PIL import Image
size = 512, 512
files=os.listdir("C:\\Users\\amita.bajaj\\Desktop\\water_semantic_segmenation\\new_data_aug\\images")
for file in files:
    im = Image.open("C:\\Users\\amita.bajaj\\Desktop\\water_semantic_segmenation\\new_data_aug\\images\\"+file)
    #width, height = im.size
    #if(width>=512 and height>=512):
    im = im.resize(size, Image.ANTIALIAS)
    im.save("C:\\Users\\amita.bajaj\\Desktop\\water_semantic_segmenation\\new_data_aug\\resized\\images\\"+str(file.split(".")[0])+".png", "PNG")

Renaming the files to follow a particular format

In [ ]:
#Renaming the files
import os

directory = 'C:\\Users\\amita.bajaj\\Desktop\\water_semantic_segmenation\\new_data_aug\\images'

# List all the files in the directory
files = os.listdir(directory)

# Iterate through each file in the directory
for index, file in enumerate(files):
    # Check if the file is an image file
    if file.endswith('.jpg') or file.endswith('.png'):
        # Construct the new file name
        new_file_name = f'{index}-water.jpg'
        # Rename the file
        os.rename(os.path.join(directory, file), os.path.join(directory, new_file_name))


Randomly shuffling the Data and splitting into Train Test and Validation Sets


In [ ]:
import os, random, shutil

#Prompting user to enter number of files to select randomly along with directory
source=input("Enter the Source Directory : ")
dest=input("Enter the Destination Directory : ")
no_of_files=int(input("Enter The Number of Files To Select : "))

print("%"*25+"{ Details Of Transfer }"+"%"*25)
print("\n\nList of Files Moved to %s :-"%(dest))

#Using for loop to randomly choose multiple files
for i in range(no_of_files):
    #Variable random_file stores the name of the random file chosen
    random_file=random.choice(os.listdir(source))
    print("%d} %s"%(i+1,random_file))
    source_file="%s/%s"%(source,random_file)
    dest_file=dest
    #"shutil.move" function moves file from one directory to another
    shutil.move(source_file,dest_file)

print("\n\n"+"$"*33+"[ Files Moved Successfully ]"+"$"*33)

Converting masks files into Json Format 



In [ ]:
import glob
import json
import os

import cv2

#from src.create_annotations import (create_image_annotation, create_annotation_format, find_contours,
                                    #get_coco_json_format, create_category_annotation)

# Label ids of the dataset
category_ids = {"water": 1}

MASK_EXT = 'png'
ORIGINAL_EXT = 'png'

In [ ]:

import cv2

image_id = 0


def find_contours(sub_mask):
    gray = cv2.cvtColor(sub_mask, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]


def create_category_annotation(category_dict):
    category_list = []
    for key, value in category_dict.items():
        category = {"id": value, "name": key, "supercategory": key}
        category_list.append(category)
    return category_list


def create_image_annotation(file_name, width, height):
    global image_id
    image_id += 1
    return {
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": file_name,
    }


def create_annotation_format(contour, image_id_, category_id, annotation_id):
    return {
        "iscrowd": 0,
        "id": annotation_id,
        "image_id": image_id_,
        "category_id": category_id,
        "bbox": cv2.boundingRect(contour),
        "area": cv2.contourArea(contour),
        "segmentation": [contour.flatten().tolist()],
    }


def get_coco_json_format():
    return {
        "info": {},
        "licenses": [],
        "images": [{}],
        "categories": [{}],
        "annotations": [{}],
    }


In [ ]:
# Get "images" and "annotations" info
def images_annotations_info(maskpath):
    annotation_id = 0
    annotations = []
    images = []

    for category in category_ids.keys():
        for mask_image in glob.glob(os.path.join(maskpath, category, f'*.{MASK_EXT}')):
            original_file_name = f'{os.path.basename(mask_image).split(".")[0]}.{ORIGINAL_EXT}'
            mask_image_open = cv2.imread(mask_image)
            height, width, c = mask_image_open.shape
            print(mask_image)

            if original_file_name not in map(lambda img: img['file_name'], images):
                image = create_image_annotation(file_name=original_file_name, width=width, height=height)
                images.append(image)
            else:
                image = [element for element in images if element['file_name'] == original_file_name][0]

            contours = find_contours(mask_image_open)

            for contour in contours:
                annotation = create_annotation_format(contour, image['id'], category_ids[category], annotation_id)
                if annotation['area'] > 0:
                    annotations.append(annotation)
                    annotation_id += 1

    return images, annotations, annotation_id


In [ ]:
#Create annotations for images
if __name__ == "__main__":
    coco_format = get_coco_json_format()  # Get the standard COCO JSON format
    
    for keyword in ["valid", "train"]:
        mask_path = f"dataset/{keyword}_mask/"
        print(mask_path)
 
        print("mask path",mask_path)
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)

        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

        with open(f"dataset/{keyword}.json", "w") as outfile:
            json.dump(coco_format, outfile, sort_keys=True, indent=4)

        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))
        
        
    